<a href="https://www.kaggle.com/code/brajeshkumr/getting-started-with-titanic?scriptVersionId=146250495" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/testdata1/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


### Got the best score with Kernel SVM (accuracy=80%)


In [2]:
train_dataset=pd.read_csv("/kaggle/input/titanic/train.csv")
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_dataset=pd.read_csv("/kaggle/input/titanic/test.csv")
test_dataset.head()
len(test_dataset)

418

#### Drop columns not required

In [4]:
#Training Dataset
train_dataset=train_dataset.drop("PassengerId", axis=1)
train_dataset=train_dataset.drop("Name", axis=1)
train_dataset=train_dataset.drop("Ticket", axis=1)
train_dataset=train_dataset.drop("Cabin", axis=1)

#Test Dataset

test_dataset=test_dataset.drop("Name", axis=1)
test_dataset=test_dataset.drop("Ticket", axis=1)
test_dataset=test_dataset.drop("Cabin", axis=1)

In [5]:
train_dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
test_dataset.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [7]:
#rearrange test dataset column

test_dataset=test_dataset.iloc[:,[0,2,7,1,3,4,5,6]].values
test_dataset

array([[892, 'male', 'Q', ..., 0, 0, 7.8292],
       [893, 'female', 'S', ..., 1, 0, 7.0],
       [894, 'male', 'Q', ..., 0, 0, 9.6875],
       ...,
       [1307, 'male', 'S', ..., 0, 0, 7.25],
       [1308, 'male', 'S', ..., 0, 0, 8.05],
       [1309, 'male', 'C', ..., 1, 1, 22.3583]], dtype=object)

#### separate features and output columns in training dataset

In [8]:
X=train_dataset.iloc[:,1:]
y=train_dataset.iloc[:,0].values

In [9]:
#adjust columns for training set

X=X.iloc[:,[1,6,0,2,3,4,5]].values
X

array([['male', 'S', 3, ..., 1, 0, 7.25],
       ['female', 'C', 1, ..., 1, 0, 71.2833],
       ['female', 'S', 3, ..., 0, 0, 7.925],
       ...,
       ['female', 'S', 3, ..., 1, 2, 23.45],
       ['male', 'C', 1, ..., 0, 0, 30.0],
       ['male', 'Q', 3, ..., 0, 0, 7.75]], dtype=object)

#### Adjust missing data

In [10]:

from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
#adjust in training dataset 
imputer.fit(X[:,2:])
X[:,2:]=imputer.transform(X[:,2:])

#adjust in test dataset

imputer.fit(test_dataset[:,3:])
test_dataset[:,3:]=imputer.transform(test_dataset[:,3:])
test_dataset

array([[892, 'male', 'Q', ..., 0.0, 0.0, 7.8292],
       [893, 'female', 'S', ..., 1.0, 0.0, 7.0],
       [894, 'male', 'Q', ..., 0.0, 0.0, 9.6875],
       ...,
       [1307, 'male', 'S', ..., 0.0, 0.0, 7.25],
       [1308, 'male', 'S', ..., 0.0, 0.0, 8.05],
       [1309, 'male', 'C', ..., 1.0, 1.0, 22.3583]], dtype=object)

#### Encode Categorical data

In [11]:
#Encode the independent features of Training Data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct_1=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])], remainder='passthrough')

X=np.array(ct_1.fit_transform(X))

ct_2=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[2])], remainder='passthrough')
X=np.array(ct_2.fit_transform(X))


#Encode the independent features of Test Data
ct_test_1=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])], remainder='passthrough')

test_dataset=np.array(ct_test_1.fit_transform(test_dataset))
ct_test_2=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[3])], remainder='passthrough')
test_dataset=np.array(ct_test_2.fit_transform(test_dataset))
print("Test set----")
print(test_dataset[2])



Test set----
[0.0 1.0 0.0 0.0 1.0 894 2.0 62.0 0.0 0.0 9.6875]


#### Splitting the Training dataset into the Training set and Test set, test set to test the model

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

print(X_train,y_train)

[[0.0 0.0 1.0 ... 0.0 0.0 7.8958]
 [0.0 0.0 1.0 ... 4.0 2.0 7.925]
 [0.0 0.0 1.0 ... 1.0 0.0 16.1]
 ...
 [0.0 1.0 0.0 ... 0.0 0.0 7.7333]
 [0.0 0.0 1.0 ... 1.0 0.0 17.4]
 [0.0 0.0 1.0 ... 1.0 1.0 39.0]] [0 1 0 0 1 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0
 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0
 0 1 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0 0 1 0 1 0 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 0
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0
 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0
 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 1 0 0 0 0 0
 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 1 0 0
 0 0 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0
 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 

#### Feature Scaling on numerical data

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#scale training data set
X_train[:,6:] = sc.fit_transform(X_train[:,6:])
X_test[:,6:] = sc.transform(X_test[:,6:])

#scale test data set

test_dataset[:,6:] = sc.transform(test_dataset[:,6:])
X_train

array([[0.0, 0.0, 1.0, ..., -0.4603716064957461, -0.4772099555405323,
        -0.4838077265875083],
       [0.0, 0.0, 1.0, ..., 2.9853228824584095, 1.956196535307373,
        -0.48323058429502463],
       [0.0, 0.0, 1.0, ..., 0.40105201574279287, -0.4772099555405323,
        -0.32165050754659447],
       ...,
       [0.0, 1.0, 0.0, ..., -0.4603716064957461, -0.4772099555405323,
        -0.4870195629754741],
       [0.0, 0.0, 1.0, ..., 0.40105201574279287, -0.4772099555405323,
        -0.2959558164428686],
       [0.0, 0.0, 1.0, ..., 0.40105201574279287, 0.7394932898834203,
        0.13097135881903862]], dtype=object)

#### Training the Kernel SVM model on the Training set

In [14]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

SVC(random_state=0)

#### Making the Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score
#Predic test records from training data set
y_pred = classifier.predict(X_test)

# print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

#Predict test records from test data set.
y_pred_test_data=classifier.predict(test_dataset)

[[120  19]
 [ 25  59]]


#### Creating a submission.csv

In [16]:
output=pd.DataFrame({'PassengerId':test_dataset[3][5], 'Survived': y_pred_test_data})
output.to_csv('submission.csv', index=False)

print('Your submission was successfully saved!')

Your submission was successfully saved!


# If you like this code then please upvote